## Align Depth to Color

In [2]:
# Fist import the library.
import pyrealsense2 as rs
import numpy as np
import cv2
import time, traceback
from PIL import Image

In [3]:
# Create a pipeline
pipeline = rs.pipeline()

In [4]:
# Create a config and configure the pipeline to stream.
# Different resolutions of color and depth streams.

config = rs.config()

In [5]:
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 60)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)

In [6]:
# Start streaming
profile = pipeline.start(config)

In [7]:
# Getting the depth sensor's depth scale.
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: ", depth_scale)

Depth Scale is:  0.0010000000474974513


### Remove the background of objects
If it's more than objectes's clipping_distance_in_meters meters away.

In [ ]:
clipping_distance_in_meters = 1 # 1 meter.
clipping_distance = clipping_distance_in_meters / depth_scale

### Create an align object
rs.align allows up to perform alignment of depth frame to others frames.

The "align_to" is the stream type to which we plan to algn depth frame.

In [10]:
align_to = rs.stream.color
align = rs.align(align_to)

In [11]:
frames = pipeline.wait_for_frames()

In [ ]:
# Align the depth frame to color frame.
aligned_frames = align.process(frames)

In [ ]:
# Get aligend frame.
aligened_depth_frame = aligned_frames.get_depth_frame()
color_frame = aligned_frames.get_color_frame()

In [ ]:
# Validate that both frame are valid.
if not aligened_depth_frame or not color_frame:
    print('The frames have unvalid one.')

In [ ]:
depth_image = np.asanyarray(aligened_depth_frame.get_data())
color_image = np.asanyarray(color_frame.get_data())

### Remove background

In [ ]:
# Remove background - Set pixels further than clipping_distance to grey.
grey_color = 153
depth_image_3d = np.dstack((depth_image, depth_image, depth_image))
# np.where(condition, [x, y]) if condition, return x, else return y.
bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

### Render images

In [ ]:
depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
images = np.hstack((bg_removed, depth_colormap))
cv2.namedWindow('Align Test', cv2.WINDOW_AUTOSIZE)
cv2.imshow('Align Test', images)

'''
key = cv2.waitKey(1)
if key & 0xFF == ord('q') or key == 27:
    cv2.destroyAllWindows()
'''

In [ ]:
# pipeline.stop()

### Streaming Loop

In [ ]:
try:
    while True:
        frames = pipeline.wait_for_frames()
        
        aligned_frames = align.process(frames)
        aligened_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        
        if not aligened_depth_frame or not color_frame:
            continue
            
        depth_image = np.asanyarray(aligened_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        #depth_image = cv2.convertScaleAbs(depth_image, alpha=0.03)
        #depth_image_255 = (255/(2**16)) * depth_image
        #depth_image_3d = np.dstack((depth_image, depth_image, depth_image))
        '''
        grey_color = 153
        depth_image_3d = np.dstack((depth_image, depth_image, depth_image))
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((bg_removed, depth_colormap))
        cv2.namedWindow('Align Test', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Test', images)
        '''
        
        #images = np.hstack((depth_image_3d, color_image))
        # cv2.namedWindow('Align Test', cv2.WINDOW_AUTOSIZE)
        # cv2.imshow('Align Test', depth_image)
        cv2.imwrite("/home/commaai-03/image/depth/%.9f.pgm" % time.time(), depth_image)
        cv2.imwrite("/home/commaai-03/image/color/%.9f.jpg" % time.time(), color_image)
        print("Shape:  ", depth_image.shape)
        print("Max: ", np.max(depth_image))
        print("Min: ", np.min(depth_image))
        print("")
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    #pipeline.stop()
    pass

### Compara two frame

In [8]:
def cal_diff(image, hash_size=8):
    width = 9
    height = 8
    # Reshape image to 9*8 and grey it.
    image = Image.fromarray(image)
    img = image.resize((width, height))
    gray_image = img.convert('L')
    pixels = list(gray_image.getdata())
    difference = []
    for h in range(height):
        start_index = h * width
        for w in range(width - 1):
            left_pixel_index = start_index + w
            difference.append(pixels[left_pixel_index] > pixels[left_pixel_index + 1])
    return difference

def hamming_distance(first, second):
    
    hamming_distance = 0
    first_diff = cal_diff(first)
    second_diff = cal_diff(second)
    
    for index, img_1 in enumerate(first_diff):
        img_2 = second_diff[index]
        if img_1 != img_2:
            hamming_distance += 1
    return hamming_distance

In [12]:
background = None

try:
    while True:
        frames = pipeline.wait_for_frames()
        
        aligned_frames = align.process(frames)
        aligened_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        
        if not aligened_depth_frame or not color_frame:
            continue
            
        depth_image = np.asanyarray(aligened_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        if background is None:
            background = color_image
            continue
        distance = hamming_distance(background, color_image)
        print("[Distance]: %s" % distance)
        
        background = color_image
        
        if distance > 5:
            cv2.imwrite("/home/commaai-03/image/depth/%.9f.pgm" % time.time(), depth_image)
            cv2.imwrite("/home/commaai-03/image/color/%.9f.jpg" % time.time(), color_image)
            print("Shape:  ", depth_image.shape)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
except Exception as e:
    traceback.print_exc()

[Distance]: 4
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 4
[Distance]: 9
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 12
Shape:   (480, 640)
[Distance]: 0
[Distance]: 9
Shape:   (480, 640)
[Distance]: 13
Shape:   (480, 640)
[Distance]: 7
Shape:   (480, 640)
[Distance]: 8
Shape:   (480, 640)
[Distance]: 4
[Distance]: 0
[Distance]: 5
[Distance]: 8
Shape:   (480, 640)
[Distance]: 0
[Distance]: 12
Shape:   (480, 640)
[Distance]: 10
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 5
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 2
[Distance]: 0
[Distance]: 5
[Distance]: 10
Shape:   (480, 640)
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 2
[Dist

[Distance]: 0
[Distance]: 4
[Distance]: 3
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 4
[Distance]: 5
[Distance]: 7
Shape:   (480, 640)
[Distance]: 5
[Distance]: 0
[Distance]: 9
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 2
[Distance]: 9
Shape:   (480, 640)
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 2
[Distance]: 4
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 

Shape:   (480, 640)
[Distance]: 0
[Distance]: 4
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 3
[Distance]: 3
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 4
[Distance]: 0
[Distance]: 1
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 3
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 6
Shape:   (480, 640)
[Distance]: 6
Shape:   (480, 640)
[Distance]: 

[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 3
[Distance]: 4
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 0
[Distance]: 4
[Distance]: 1
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 2
[Distance]: 1
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distanc

[Distance]: 10
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 3
[Distance]: 4
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 5
[Distance]: 1
[Distance]: 6
Shape:   (480, 640)
[Distance]: 4
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 8
Shape:   (480, 640)
[Distance]: 5
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 2
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 4
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 5
[Distance]: 5
[Distance]: 3
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 0

...

[Distance]: 2
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 4
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 0
[Distance]: 1
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Dista

[Distance]: 0
[Distance]: 5
[Distance]: 4
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 4
[Distance]: 6
Shape:   (480, 640)
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 8
Shape:   (480, 640)
[Distance]: 5
[Distance]: 4
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 1
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 4
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[D

[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 10
Shape:   (480, 640)
[Distance]: 0
[Distance]: 9
Shape:   (480, 640)
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 4
[Distance]: 4
[Distance]: 0
[Distance]: 5
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 9
Shape:   (480, 640)
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 3
[Distance]: 5
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 9
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3

[Distance]: 0
[Distance]: 5
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 2
[Distance]: 0
[Distance]: 4
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 5
[Distance]: 0
[Distance]: 8
Shape:   (480, 640)
[Distance]: 5
[Distance]: 4
[Distance]: 0
[Distance]: 6
Shape:   (480, 640)
[Distance]: 4
[Distance]: 4
[Distance]: 0
[Distance]: 7
Shape:   (480, 640)
[Distance]: 0
[Distance]: 3
[Distance]: 0
[Distance]: 1
[Distance]: 0
[Distance]: 2
[D

KeyboardInterrupt: 